### District Summary

* Create a high level snapshot (in table form) of the district's key metrics, including:
  * Total Schools
  * Total Students
  * Total Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [1]:
import pandas as pd
import os

In [2]:
dataPath = os.path.join('.','PyCitySchools', 'raw_data')

In [330]:
school = pd.read_csv(os.path.join(dataPath, 'schools_complete.csv'))
student = pd.read_csv(os.path.join(dataPath, 'students_complete.csv'))
student['reading_pass']= student.apply(lambda x : 'pass' if int(x['reading_score']) > 60 else 'Fail', axis=1)
student['math_pass']= student.apply(lambda x : 'pass' if int(x['math_score']) > 60 else 'Fail', axis=1)
#student.groupby('school').count()
student.head()

,Student ID,name,gender,grade,school,reading_score,math_score,reading_pass,math_pass
0,0,Paul Bradley,M,9th,Huang High School,66,79,pass,pass
1,1,Victor Smith,M,12th,Huang High School,94,61,pass,pass
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,pass,Fail
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,pass,Fail
4,4,Bonnie Ray,F,9th,Huang High School,97,84,pass,pass


In [162]:


summaryDF = pd.DataFrame({'Total Schools': [school['name'].count()],\
                          'Total Students': [student['name'].count()],\
                          'Total Budget': ['${:,.2f}'.format(school['budget'].sum())],
                          'Average Math Score': [round(student['math_score'].mean(), 2)],\
                          'Average Reading Score': [round(student['reading_score'].mean(), 2)],\
                          'Percent_Passing_Math': ["{:.2%}".format(student.loc[student['math_pass']=='pass'].count().iat[0]/student['name'].count())],\
                          'Percent_Passing_Reading': ["{:.2%}".format(student.loc[student['reading_pass']=='pass'].count().iat[0]/student['name'].count())],\
                          'Overall Passing Rate': ["{:.2%}".format((student.loc[student['reading_pass']=='pass'].count().iat[0]/student['name'].count()+\
                                                                   student.loc[student['math_pass']=='pass'].count().iat[0]/student['name'].count())/2)]})
summaryDF


,Average Math Score,Average Reading Score,Overall Passing Rate,Percent_Passing_Math,Percent_Passing_Reading,Total Budget,Total Schools,Total Students
0,78.99,81.88,95.45%,90.91%,100.00%,"$24,649,428.00",15,39170


### School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [366]:
#Score Averages
school = pd.read_csv(os.path.join(dataPath, 'schools_complete.csv'))
school = school.set_index('name')
schoolAverages = student.groupby('school').mean()
schoolAverages = schoolAverages.loc[:,'reading_score':'math_score'].rename(columns={'reading_score':'ave_reading_score', 'math_score':'ave_math_score'})
school = pd.merge(school, schoolAverages, left_index=True, right_index=True)
#Student totals
schoolTotals = student.groupby('school').count()
schoolTotals = schoolTotals.loc[:,'name'].rename(columns={'name':'Total Students'})
school['Total Students'] = schoolTotals
#per student budget
school['Per Student Budget'] = school.apply(lambda x : '${:,.2f}'.format(x['budget']/x['Total Students']), axis=1)
#%passing Math
percentMathPass = student.loc[student['math_pass']=='pass'].groupby(['school']).count()/student.groupby(['school']).count()
percentMathPass = percentMathPass.loc[:,'Student ID'].rename(columns={'Student ID': 'Percent passing math'})
school['Percent passing math'] = round(percentMathPass*100,2) 
school
#%passing Reading
percentPass = student.loc[student['reading_pass']=='pass'].groupby(['school']).count()/student.groupby(['school']).count()
percentPass = percentPass.loc[:,'Student ID'].rename(columns={'Student ID': 'Percent passing reading'})
school['Percent passing reading'] = round(percentPass*100,2) 
school['Average Pass Rate']  = (school['Percent passing reading'] + school['Percent passing math'])/2
school


,School ID,type,size,budget,ave_reading_score,ave_math_score,Total Students,Per Student Budget,Percent passing math,Percent passing reading,Average Pass Rate
Huang High School,0,District,2917,1910635,81.182722,76.629414,2917,$655.00,86.84,100.0,93.420
Figueroa High School,1,District,2949,1884411,81.158020,76.711767,2949,$639.00,86.44,100.0,93.220
Shelton High School,2,Charter,1761,1056600,83.725724,83.359455,1761,$600.00,100.00,100.0,100.000
Hernandez High School,3,District,4635,3022020,80.934412,77.289752,4635,$652.00,86.45,100.0,93.225
Griffin High School,4,Charter,1468,917500,83.816757,83.351499,1468,$625.00,100.00,100.0,100.000
Wilson High School,5,Charter,2283,1319574,83.989488,83.274201,2283,$578.00,100.00,100.0,100.000
Cabrera High School,6,Charter,1858,1081356,83.975780,83.061895,1858,$582.00,100.00,100.0,100.000
Bailey High School,7,District,4976,3124928,81.033963,77.048432,4976,$628.00,87.44,100.0,93.720
Holden High School,8,Charter,427,248087,83.814988,83.803279,427,$581.00,100.00,100.0,100.000
Pena High School,9,Charter,962,585858,84.044699,83.839917,962,$609.00,100.00,100.0,100.000


# Heroes
Congratulations! After a lot of hard work in the data munging mines, you've landed a job as Lead Analyst for an independent gaming company. You've been assigned the task of analyzing the data for their most recent fantasy game Heroes of Pymoli.

Like many others in its genre, the game is free-to-play, but players are encouraged to purchase optional items that enhance their playing experience. As a first task, the company would like you to generate a report that breaks down the game's purchasing data into meaningful insights.

Your final report should include each of the following:

Player Count
Total Number of Players
Purchasing Analysis (Total)
Number of Unique Items
Average Purchase Price
Total Number of Purchases
Total Revenue
Gender Demographics
Percentage and Count of Male Players
Percentage and Count of Female Players
Percentage and Count of Other / Non-Disclosed
Purchasing Analysis (Gender)
##The below each broken by gender
Purchase Count
Average Purchase Price
Total Purchase Value
Normalized Totals
Age Demographics
##The below each broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.)
*Purchase Count
*Average Purchase Price
*Total Purchase Value
*Normalized Totals
*Top Spenders
*Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
*SN
*Purchase Count
*Average Purchase Price
*Total Purchase Value
*Most Popular Items
*Identify the 5 most popular items by purchase count, then list (in a table):
*Item ID
*Item Name
*Purchase Count
*Item Price
*Total Purchase Value
*Most Profitable Items
*Identify the 5 most profitable items by total purchase value, then list (in a table):
*Item ID
*Item Name
*Purchase Count
*Item Price
*Total Purchase Value
##As final considerations:

You must use the Pandas Library and the Jupyter Notebook.
You must submit a link to your Jupyter Notebook with the viewable Data Frames.
You must include an exported markdown version of your Notebook called  README.md in your GitHub repository.
You must include a written description of three observable trends based on the data.
See Example Solution for a reference on expected format.


In [371]:
import os
import pandas as pd
dataPath = os.path.join('.','HeroesOfPymoli')
numbers = pd.read_json(os.path.join(dataPath, 'purchase_data.json'))
numbers.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [376]:
### Player Count

#* Total Number of Players
numbers['SN'].nunique()

573

In [389]:
### Purchasing Analysis (Total)

#* Number of Unique Items
items = numbers['Item ID'].nunique()
#* Average Purchase Price
avePrice = numbers['Price'].mean()
#* Total Number of Purchases
totalP = numbers.shape[0]
#* Total Revenue
rev = numbers['Price'].sum()
PAnalysis = pd.DataFrame({'N of Unique Items': [items], 'Average Purchase Prices': [avePrice], 'Total Purchases': [totalP], 'Revenue': [rev]})
PAnalysis


,Average Purchase Prices,N of Unique Items,Revenue,Total Purchases
0,2.931192,183,2286.33,780


In [408]:
### Gender Demographics

#* Percentage and Count of Male Players
perMale = round(numbers.loc[numbers['Gender']=='Male'].shape[0]/numbers.shape[0]*100,2)
perMale
#* Percentage and Count of Female Players
perF = round(numbers.loc[numbers['Gender']=='Female'].shape[0]/numbers.shape[0]*100,2)
perF
#* Percentage and Count of Other / Non-Disclosed
uk = round(100-perMale-perF,2)
uk
print(pd.DataFrame({'Percent Male': [perMale], 'Percent Female': [perF], 'Undisclosed':[uk]}))

   Percent Female  Percent Male  Undisclosed
0           17.44         81.15         1.41


In [442]:
numbers.groupby('Gender').sum()#.loc['Female','Price']

,Age,Item ID,Price
Gender,,,
Female,3068,11983,382.91
Male,14360,57965,1867.68
Other / Non-Disclosed,301,1261,35.74


In [496]:
### Purchasing Analysis (Gender)

#* The below each broken by gender
byGender = {}; byGender['Purchase Count'] = {};byGender['Average Purchase Price'] = {}; byGender['Purchase Value'] = {}
for index in numbers.groupby('Gender').count().index:
#  * Purchase Count
    byGender['Purchase Count'][index] = numbers.groupby('Gender').count().loc[index,'Age']
#  * Average Purchase Price
    byGender['Average Purchase Price'][index] = round(numbers.groupby('Gender').mean().loc[index,'Price'],2)
#  * Total Purchase Value
    byGender['Purchase Value'][index] = numbers.groupby('Gender').sum().loc[index,'Price']
    
#  * Normalized Totals
byGender = pd.DataFrame(byGender)
byGender


,Average Purchase Price,Purchase Count,Purchase Value
Female,2.82,136,382.91
Male,2.95,633,1867.68
Other / Non-Disclosed,3.25,11,35.74


In [499]:
### Age Demographics

#  * The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
AgeDems = pd.DataFrame({})
# Create the bins in which Data will be held
bins = range(0,100,4)
# Create the names for the four bins
names = [str(Bin)+'-'+str(Bin+3) for Bin in bins][0:-1]
numbers['Age Group'] = pd.cut(numbers["Age"], bins, labels=names)
byAge = {}; byAge['Purchase Count'] = {};byAge['Average Purchase Price'] = {}; byAge['Purchase Value'] = {}
for index in names:
#  * Purchase Count
    byAge['Purchase Count'][index] = numbers.groupby('Age Group').count().loc[index,'Age']
#  * Average Purchase Price
    byAge['Average Purchase Price'][index] = round(numbers.groupby('Age Group').mean().loc[index,'Price'],2)
#  * Total Purchase Value
    byAge['Purchase Value'][index] = numbers.groupby('Age Group').sum().loc[index,'Price']
    
#  * Normalized Totals
byAge = pd.DataFrame(byAge)
byAge

,Average Purchase Price,Purchase Count,Purchase Value
0-3,NaN,0,0.00
12-15,2.75,87,238.89
16-19,2.91,161,468.03
20-23,2.92,238,696.09
24-27,2.97,104,309.37
28-31,3.06,66,202.09
32-35,2.98,38,113.28
36-39,2.90,37,107.35
4-7,2.79,22,61.34
40-43,2.96,2,5.92
